# Data

In [4]:
import pandas as pd

df = pd.read_csv('data/cleaned_전국공장등록현황_preprocessed_seoul.csv')
df

/var/folders/3k/09hbppyx33z8tmqmvw42bgqm0000gn/T/ipykernel_77710/4106736577.py:3: DtypeWarning: Columns (11,13,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/cleaned_전국공장등록현황_preprocessed_seoul.csv')


,__row_uid__,시도명,시군구명,관리기관,회사명,공장구분,단지명,설립구분,입주형태,보유구분,...,정제_용도지역,코드매핑_용도지역,누적_용도지역,정제_지목,코드매핑_지목,누적_지목,정제_최초등록일,누적_최초등록일,정제_최초승인일,누적_최초승인일
0,0_9c80b0bc,서울특별시,종로구,서울특별시 종로구,남일문화 (주),개별입지,,일반,해당없음,임대,...,도시지역,R10-UQA001,{'도시지역': ['R10-UQA001']},대,R70-08,{'대': ['R70-08']},2005-02-02,{'2005-02-02': ['']},NaN,{}
1,0_9c80b0bc,서울특별시,종로구,서울특별시 종로구,남일문화 (주),개별입지,,일반,해당없음,임대,...,주거지역,R10-UQA100,{'주거지역': ['R10-UQA100']},대,R70-08,{'대': ['R70-08']},2005-02-02,{'2005-02-02': ['']},NaN,{}
2,0_9c80b0bc,서울특별시,종로구,서울특별시 종로구,남일문화 (주),개별입지,,일반,해당없음,임대,...,제2종일반주거지역,R10-UQA122,{'제2종일반주거지역': ['R10-UQA122']},대,R70-08,{'대': ['R70-08']},2005-02-02,{'2005-02-02': ['']},NaN,{}
3,1_386f6646,서울특별시,종로구,서울특별시 종로구,(주)에취.알.디,개별입지,,일반,해당없음,임대,...,도시지역,R10-UQA001,{'도시지역': ['R10-UQA001']},대,R70-08,{'대': ['R70-08']},1994-03-07,{'1994-03-07': ['']},1998-01-01,{'1998-01-01': ['']}
4,1_386f6646,서울특별시,종로구,서울특별시 종로구,(주)에취.알.디,개별입지,,일반,해당없음,임대,...,주거지역,R10-UQA100,{'주거지역': ['R10-UQA100']},대,R70-08,{'대': ['R70-08']},1994-03-07,{'1994-03-07': ['']},1998-01-01,{'1998-01-01': ['']}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33091,11224_2b8fcc16,서울특별시,강동구,서울특별시 강동구,한국가죽산업협동조합,개별입지,,일반,일반건축물,임대,...,주거지역,R10-UQA100,{'주거지역': ['R10-UQA100']},대,R70-08,{'대': ['R70-08']},2015-03-20,{'2015-03-20': ['']},NaN,{}
33092,11224_2b8fcc16,서울특별시,강동구,서울특별시 강동구,한국가죽산업협동조합,개별입지,,일반,일반건축물,임대,...,제2종일반주거지역,R10-UQA122,{'제2종일반주거지역': ['R10-UQA122']},대,R70-08,{'대': ['R70-08']},2015-03-20,{'2015-03-20': ['']},NaN,{}
33093,11225_77223828,서울특별시,강동구,서울특별시 강동구,한올EMB,개별입지,,일반,일반건축물,임대,...,도시지역,R10-UQA001,{'도시지역': ['R10-UQA001']},대,R70-08,{'대': ['R70-08']},2015-03-11,{'2015-03-11': ['']},NaN,{}
33094,11225_77223828,서울특별시,강동구,서울특별시 강동구,한올EMB,개별입지,,일반,일반건축물,임대,...,주거지역,R10-UQA100,{'주거지역': ['R10-UQA100']},대,R70-08,{'대': ['R70-08']},2015-03-11,{'2015-03-11': ['']},NaN,{}


## 서울시 연령별/동별 통계

* https://data.seoul.go.kr/dataList/10718/S/2/datasetView.do

In [24]:
df_age = pd.read_excel('data/등록인구(연령별_동별)_20251130135503.xlsx')
df_age = df_age.loc[(df_age['항목']=='계') & (df_age['동별(1)']!='합계')][['동별(1)', '2025 2/4']]

df_age.rename(columns={'동별(1)':'시군구명', '2025 2/4':'인구'}, inplace=True)
df_age.reset_index(inplace=True, drop=True)
df_age

/Users/jeongyunl/Documents/GitHub/factory-chatbot-demo/env/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,시군구명,인구
0,종로구,149332
1,중구,129616
2,용산구,216214
3,성동구,280683
4,광진구,350164
5,동대문구,361871
6,중랑구,383958
7,성북구,437692
8,강북구,287523
9,도봉구,304555


## 서울시 행정구역(구별) 통계

* https://data.seoul.go.kr/dataList/412/S/2/datasetView.do

In [25]:
df_area = pd.read_excel('data/행정구역(구별)_20251130135354.xlsx', header=2)
df_area = df_area.loc[df_area['자치구별(2)']!='소계'][['자치구별(2)', '면적 (km²)']]
df_area.rename(columns={'자치구별(2)':'시군구명', '면적 (km²)':'면적'}, inplace=True)
df_area.reset_index(inplace=True, drop=True)
df_area




/Users/jeongyunl/Documents/GitHub/factory-chatbot-demo/env/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,시군구명,면적
0,종로구,23.91
1,중구,9.96
2,용산구,21.87
3,성동구,16.86
4,광진구,17.06
5,동대문구,14.22
6,중랑구,18.50
7,성북구,24.57
8,강북구,23.60
9,도봉구,20.65


## 통합

In [21]:
# '누적_'으로 시작하는 컬럼들 삭제
df.drop(columns=[col for col in df.columns if col.startswith('누적_')], inplace=True)

# '코드매핑_'으로 시작하는 컬럼들 삭제
df.drop(columns=[col for col in df.columns if col.startswith('코드매핑_')], inplace=True)

In [38]:
df.columns

Index(['__row_uid__', '시도명', '시군구명', '관리기관', '회사명', '공장구분', '단지명', '설립구분',
       '입주형태', '보유구분', '최초승인일', '최초등록일', '등록구분', '등록일', '전화번호', '남자종업원',
       '여자종업원', '외국인남자종업원', '외국인여자종업원', '종업원합계', '생산품', '원자재', '공장규모', '용도지역',
       '지목', '용지면적', '제조시설면적', '부대시설면적', '건축면적', '지식산업센터명', '대표업종', '업종명',
       '업종코드', '차수', '법인주소', '필지수', '공장주소', '공장주소_지번', '공장관리번호', '정제_공장주소',
       '정제_공장주소_지번', '정제_관리기관', '정제_대표업종', '정제_등록일', '정제_법인주소', '정제_보유구분',
       '정제_시군구명', '정제_시도명', '정제_업종명', '정제_업종코드', '정제_용도지역', '정제_지목',
       '정제_최초등록일', '정제_최초승인일'],
      dtype='object')

In [32]:
df['정제_시군구명'].nunique()

25

In [40]:
df_total = pd.merge(df, df_age, left_on='정제_시군구명', right_on='시군구명', how='left')
df_total = pd.merge(df_total, df_area, left_on='정제_시군구명', right_on='시군구명', how='left')

df_total.drop(columns=['시군구명'], inplace=True)
df_total.head()

,__row_uid__,시도명,시군구명_x,관리기관,회사명,공장구분,단지명,설립구분,입주형태,보유구분,...,정제_시도명,정제_업종명,정제_업종코드,정제_용도지역,정제_지목,정제_최초등록일,정제_최초승인일,시군구명_y,인구,면적
0,0_9c80b0bc,서울특별시,종로구,서울특별시 종로구,남일문화 (주),개별입지,,일반,해당없음,임대,...,서울특별시,경 인쇄업,"18111,18113,18119",도시지역,대,2005-02-02,NaN,종로구,149332,23.91
1,0_9c80b0bc,서울특별시,종로구,서울특별시 종로구,남일문화 (주),개별입지,,일반,해당없음,임대,...,서울특별시,경 인쇄업,"18111,18113,18119",주거지역,대,2005-02-02,NaN,종로구,149332,23.91
2,0_9c80b0bc,서울특별시,종로구,서울특별시 종로구,남일문화 (주),개별입지,,일반,해당없음,임대,...,서울특별시,경 인쇄업,"18111,18113,18119",제2종일반주거지역,대,2005-02-02,NaN,종로구,149332,23.91
3,1_386f6646,서울특별시,종로구,서울특별시 종로구,(주)에취.알.디,개별입지,,일반,해당없음,임대,...,서울특별시,전시용 모형 제조업,33932,도시지역,대,1994-03-07,1998-01-01,종로구,149332,23.91
4,1_386f6646,서울특별시,종로구,서울특별시 종로구,(주)에취.알.디,개별입지,,일반,해당없음,임대,...,서울특별시,전시용 모형 제조업,33932,주거지역,대,1994-03-07,1998-01-01,종로구,149332,23.91


In [41]:
df_total.to_csv('data/전국공장등록현황_서울_통합.csv', index=False, encoding='utf-8-sig')

# ver1

In [ ]:
import uuid, os, io, sys, time
import asyncio
import threading
from contextlib import asynccontextmanager
from dotenv import load_dotenv
from typing import TypedDict
from typing import Annotated
import pandas as pd
import numpy as np
from functools import wraps

from openai import OpenAI
from langchain_openai import ChatOpenAI
# import chromadb
# from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

from langchain_community.tools.tavily_search import TavilySearchResults

from langgraph.graph import END, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langgraph.errors import GraphRecursionError

from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableConfig  
from langchain_core.output_parsers import StrOutputParser

from langchain_community.chat_message_histories import ChatMessageHistory, StreamlitChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langgraph.graph.message import add_messages
from operator import itemgetter

from langchain.agents import tool
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

# from langchain_teddynote import logging
from langsmith import traceable
import threading

In [ ]:
# 환경변수 로드
load_dotenv(override=True)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY not found in environment variables")

# 모델
model = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model="gpt-5.1-2025-11-13",
    temperature=0
)

In [ ]:
class GraphState(TypedDict):
    question: str # 질문
    q_type: str  # 질문의 유형
    answer: str | list[str]   # llm이 생성한 답변
    session_id: str  # 세션 ID 추가

# 🔧 개선 1: 스레드 안전한 저장소
import threading
from collections import defaultdict

class ThreadSafeStore:
    def __init__(self):
        self._store = {}
        self._lock = threading.RLock()  # 재진입 가능한 락
    
    def get_session_history(self, session_id: str):
        with self._lock:
            if session_id not in self._store:
                self._store[session_id] = ChatMessageHistory()
                print(f"🆕 새로운 세션 히스토리 생성: {session_id[:8]}...")
            return self._store[session_id]
    
    def clear_session(self, session_id: str = None):
        with self._lock:
            if session_id:
                if session_id in self._store:
                    message_count = len(self._store[session_id].messages)
                    del self._store[session_id]
                    return message_count
                return 0
            else:
                total_sessions = len(self._store)
                total_messages = sum(len(history.messages) for history in self._store.values())
                self._store.clear()
                return total_sessions, total_messages
    
    def get_stats(self):
        with self._lock:
            return {
                'total_sessions': len(self._store),
                'total_messages': sum(len(history.messages) for history in self._store.values())
            }

# 전역 스레드 안전 저장소
thread_safe_store = ThreadSafeStore()

# 세션 ID를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    return thread_safe_store.get_session_history(session_ids)

# 새로운 세션 ID 생성 함수
def generate_session_id():
    return str(uuid.uuid4())

In [ ]:
#######################################################################
############################ nodes: Router ############################
#######################################################################

class Router(BaseModel):
    type: str = Field(description="type of the query that model choose. Choose from ['general', 'domain_specific']")

router_output_parser = JsonOutputParser(pydantic_object=Router)
router_format_instructions = router_output_parser.get_format_instructions()

router_prompt = PromptTemplate(
    template="""
            You are an expert who classifies the type of question. There are two query types: ['general', 'domain_specific']

            [general]
            Questions unrelated to data query, such as translating English to Korean, asking for general knowledge (e.g., "What is the capital of South Korea?"), or queries that can be answered through a web search.

            [domain_specific]
            Questions related to 'factory' domain and data query, such as 'count the unique values of factories in Seoul', or count 'the number of rows in a table'.

            <Output format>: Always respond with either "general" or "domain_specific" and nothing else. {format_instructions}
            <chat_history>: {chat_history}
            
            <Question>: {query} 
            """,
    input_variables=["query", "chat_history"],
    partial_variables={"format_instructions": router_format_instructions},
)

def router(state: GraphState) -> GraphState:
    chain = router_prompt | model | router_output_parser
    
    router_with_history  = RunnableWithMessageHistory(
        chain,
        get_session_history,
        input_messages_key="query",
        history_messages_key="chat_history",
    )
    
    router_result = router_with_history.invoke(
        {"query": state["question"]}, 
        {'configurable': {'session_id': state["session_id"]}}
    )
    state["q_type"] = router_result['type']
    return state

def router_conditional_edge(state: GraphState) -> GraphState:
    q_type = state["q_type"].strip()
    return q_type

In [ ]:
# 테스트 
# def router(input):
#     chain = router_prompt | model | router_output_parser
    
#     router_with_history  = RunnableWithMessageHistory(
#         chain,
#         get_session_history,
#         input_messages_key="query",
#         history_messages_key="chat_history",
#     )
    
#     router_result = router_with_history.invoke(
#         {"query": input}, 
#         {'configurable': {'session_id': '111'}}
#     )
#     return router_result

# router('서울에 구별로 몇개의 공장이 있나요?')

In [10]:
##################################################################################
###################### nodes: Generate Python Pandas Code ########################
##################################################################################

class CodeGenerator(BaseModel):
    code: str = Field(description="Python Pandas Code")

code_generator_output_parser = JsonOutputParser(pydantic_object=CodeGenerator)
code_generator_format_instructions = code_generator_output_parser.get_format_instructions()

code_generator_prompt = PromptTemplate(
    template="""
            You are an expert who can generate Python Pandas Code to answer the query.

            Write the code with the following dataset metadata. Do not use any other columns except the ones provided in the metadata. The columns are written in Korean.

            <Dataset Metadata>: 
            # Basic Information
            1. '공장관리번호' (Factory Management Number): Unique factory identification number. [Important] A single factory management number can appear across multiple rows. When counting the number of factories, always use unique/distinct values of this field.

            # Company & Factory Information
            2. '회사명' (Company Name): Name of the company operating the factory. It's not unique. 
            3. '공장구분' (Factory Classification): Type/classification of the factory
            4. '단지명' (Complex Name): Name of the industrial complex (if applicable)
            5. '설립구분' (Establishment Type): Classification of how the factory was established
            6. '입주형태' (Occupancy Type): Type of occupancy arrangement
            7. '등록구분' (Registration Type): Classification of factory registration
            8. '전화번호' (Phone Number): Contact phone number

            # Employee Statistics
            9. '남자종업원' (Male Employees): Number of male employees
            10. '여자종업원' (Female Employees): Number of female employees
            11. '외국인남자종업원' (Foreign Male Employees): Number of foreign male employees
            12. '외국인여자종업원' (Foreign Female Employees): Number of foreign female employees
            13. '종업원합계' (Total Employees): Total number of employees

            # Production Information
            14. '생산품' (Products): Products manufactured at the factory
            15. '원자재' (Raw Materials): Raw materials used in production
            16. '공장규모' (Factory Scale): Size classification of the factory
            
            # Facility Specifications
            17. '용지면적' (Land Area): Total land area in square meters
            18. '제조시설면적' (Manufacturing Facility Area): Area dedicated to manufacturing facilities
            19. '부대시설면적' (Auxiliary Facility Area): Area of auxiliary/support facilities
            20. '건축면적' (Building Area): Total building area
            21. '지식산업센터명' (Knowledge Industry Center Name): Name of knowledge industry center (if applicable)

            # Location & Administrative
            22. '필지수' (Number of Parcels): Number of land parcels
            23. '공장관리번호' (Factory Management Number): Unique factory identification number

            #Standardized Fields (정제_)
            24. '정제_관리기관' (Standardized Management Agency): Standardized name of the management agency 
            25. '정제_보유구분' (Standardized Ownership Type): Standardized ownership classification
            26. '정제_시군구명' (Standardized District Name): Standardized city/county/district name
            27. '정제_시도명' (Standardized Province Name): Standardized province/metropolitan city name
            28. '정제_업종명' (Standardized Industry Name): Standardized industry name. It's not unique, so you need to calculate with '정제_대표업종' and show in '정제_업종명'
            29. '정제_대표업종' (Standardized Primary Industry): Standardized primary industry classification. It's in code, so after use it, you need to show the name using '정제_대표업종'
            29. '정제_용도지역' (Standardized Zoning District): Standardized zoning/land use district
            30. '정제_지목' (Standardized Land Category): Standardized land category classification

            # Date Fields
            31. '정제_최초등록일' (Standardized Initial Registration Date): Standardized date of initial registration (format: YYYY-MM-DD)
            32. '정제_최초승인일' (Standardized Initial Approval Date): Standardized date of initial approval (format: YYYY-MM-DD)

            Write the code with the most efficient way.
            <Output format>: Always respond with Python Pandas code. Always assign the final result to a variable called `return_var`. Do not use print(). {format_instructions}
            <chat_history>: {chat_history}
            
            <Question>: {query} 
            """,
    input_variables=["query", "chat_history"],
    partial_variables={"format_instructions": code_generator_format_instructions},
)

# def query_generator(state: GraphState) -> GraphState:
#     chain = query_generator_prompt | model | query_generator_output_parser
    
#     query_generator_with_history  = RunnableWithMessageHistory(
#         chain,
#         get_session_history,
#         input_messages_key="query",
#         history_messages_key="chat_history",
#     )
    
#     query_generator_result = query_generator_with_history.invoke(
#         {"query": state["question"]}, 
        
#         {'configurable': {'session_id': state["session_id"]}}
#     )
#     state["code"] = query_generator_result['code']
#     return state

In [11]:
@tool
def code_generator(input):
    '''
    사용자의 질문에 답하기 위해 CSV에서 쿼리할 수 있는 Python Pandas 코드를 작성하는 도구
    '''
    chain = code_generator_prompt | model | code_generator_output_parser
    
    code_generator_with_history  = RunnableWithMessageHistory(
        chain,
        get_session_history,
        input_messages_key="query",
        history_messages_key="chat_history",
    )
    
    code_generator_result = code_generator_with_history.invoke(
        {"query": input}, 
        {'configurable': {'session_id': '111'}}
    )
    return code_generator_result['code']

In [7]:
# 테스트
answer = code_generator(' 여성 직원 비율이 높은 공장은 어떤 업종이 많아?')
print(answer)

/Users/jeongyunl/Documents/GitHub/factory-chatbot-demo/env/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


🆕 새로운 세션 히스토리 생성: 111...
import pandas as pd

# 여성 직원 비율 계산 (여자종업원 / 종업원합계)
# 0 또는 NaN으로 인한 오류를 방지하기 위해 조건부 계산
valid_emp = df['종업원합계'] > 0
female_ratio = pd.Series(0, index=df.index, dtype='float64')
female_ratio[valid_emp] = df.loc[valid_emp, '여자종업원'] / df.loc[valid_emp, '종업원합계']

# 여성 직원 비율이 높은 공장 정의: 상위 25% (사분위수 기준)
threshold = female_ratio.quantile(0.75)

# 여성 비율이 높은 공장만 필터링
high_female_df = df[female_ratio >= threshold].copy()

# 공장 단위로 중복 제거 (공장관리번호 기준)
# 동일 공장이 여러 행에 있을 수 있으므로, 대표 행 1개만 사용
high_female_unique = high_female_df.drop_duplicates(subset=['공장관리번호'])

# 업종별(정제_대표업종 코드 기준) 공장 수 집계
industry_counts = (
    high_female_unique
    .groupby('정제_대표업종')['공장관리번호']
    .nunique()
    .reset_index(name='여성비율_높은_공장수')
)

# 공장 수 기준으로 내림차순 정렬
industry_counts = industry_counts.sort_values('여성비율_높은_공장수', ascending=False)

# 업종명(정제_업종명)을 함께 보기 위해 대표업종 코드 기준으로 매핑
# 각 대표업종 코드당 하나의 업종명을 대표로 사용
industry_name_map = (
    df.dropna(subset=['정제_대표업종'])
      .drop_duplicates(subset=['정제_대표업종'

In [12]:
@tool
def code_executor(input_code: str):
    """
    LLM이 생성한 Pandas 코드를 실행하고 return_var 값을 반환하는 Tool.
    df는 Tool 안에서 이미 글로벌로 설정되어 있어야 함.
    """
    global df  # 이미 정의된 df를 사용
    local_vars = {'df': df}
    exec(input_code, local_vars)
    if 'return_var' not in local_vars:
        raise ValueError("Generated code did not assign value to 'return_var'.")
    return local_vars['return_var']

In [14]:
# 테스트
df = pd.read_csv('data/cleaned_전국공장등록현황_preprocessed_seoul.csv')

/var/folders/3k/09hbppyx33z8tmqmvw42bgqm0000gn/T/ipykernel_21420/2686250579.py:2: DtypeWarning: Columns (11,13,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/cleaned_전국공장등록현황_preprocessed_seoul.csv')


In [ ]:
# 테스트
answer = code_generator(' 여성 직원 비율이 높은 공장은 어떤 업종이 많아?')
print(answer)

In [15]:
@tool
def code_executor(input_code: str, max_retries=3):
    """
    LLM이 생성한 Pandas 코드를 안전하게 실행하고 return_var 반환.
    df는 글로벌 변수 사용.
    NA, None, 0 등의 에러 대비.
    """
    global df
    local_vars = {'df': df}

    for attempt in range(max_retries):
        try:
            exec(input_code, local_vars)
            if 'return_var' not in local_vars:
                raise ValueError("Generated code did not assign value to 'return_var'.")
            return local_vars['return_var']
        except Exception as e:
            print(f"⚠️ 코드 실행 실패 (시도 {attempt+1}/{max_retries}): {e}")
            # NA나 boolean 비교 에러 등 재시도 가능
            if attempt == max_retries - 1:
                raise

In [ ]:
# 테스트
df = pd.read_csv('data/cleaned_전국공장등록현황_preprocessed_seoul.csv')
result = code_executor(answer)
result

In [16]:
############################ tools & Agents ############################

# 🔧 개선 3: OpenAI API 레이트 리미팅 및 재시도
import openai
from openai import RateLimitError, APITimeoutError

def retry_on_failure(max_retries=3, delay=1):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            last_exception = None
            for attempt in range(max_retries):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    last_exception = e
                    if attempt < max_retries - 1:
                        print(f"⚠️ 시도 {attempt + 1} 실패, {delay}초 후 재시도: {str(e)[:100]}")
                        time.sleep(delay * (attempt + 1))  # 지수 백오프
                    else:
                        print(f"❌ 모든 재시도 실패: {str(e)}")
            raise last_exception
        return wrapper
    return decorator

@retry_on_failure(max_retries=3, delay=2)
def call_openai_with_retry(client, **kwargs):
    try:
        return client.chat.completions.create(**kwargs)
    except RateLimitError as e:
        print(f"⚠️ OpenAI 레이트 리미트: {e}")
        time.sleep(5)  # 레이트 리미트 시 더 오래 대기
        raise
    except APITimeoutError as e:
        print(f"⚠️ OpenAI 타임아웃: {e}")
        raise
    except Exception as e:
        print(f"⚠️ OpenAI API 오류: {e}")
        raise
    
tools = [code_generator, code_executor]

agent_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that answers ONLY in Korean. "
            "You must follow these rules:\n"
            "1. If the question's type (`q_type`) is 'domain_specific', you MUST use tools (`code_generator` and `code_executor`) to generate Python Pandas code and execute it step by step. "
            "   Always base your answer on the return_var from code execution. If the data does not allow you to answer, respond with '참조할 정보가 없어서 답변할 수 없습니다.'\n"
            "2. If the question's type is NOT 'domain_specific', do NOT use any tools. You may answer directly, but you MUST begin your answer with '데이터에 기반하지 않은 답변입니다' (This answer is not based on data).\n"
            "3. You may repeat code generation and execution multiple times if needed, but you must always base your answer on return_var when using tools. "
            "Always answer in Korean, never in English."
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("human", "{retrieved_data}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

def agent(state: GraphState) -> GraphState:
    """
    Agent 실행 함수
    - domain_specific 질문은 tools(code_generator + safe_code_executor) 사용
    - code 실행 실패 시 재시도 구조 적용
    """
    try:
        # Agent 생성
        agent_obj = create_tool_calling_agent(model, tools, agent_prompt)

        agent_executor = AgentExecutor(
            agent=agent_obj,
            tools=tools,
            verbose=False,
            max_iterations=10,
            max_execution_time=120,
            handle_parsing_errors=True,
            return_intermediate_steps=True
        )

        agent_with_history = RunnableWithMessageHistory(
            agent_executor,
            get_session_history,
            history_messages_key="chat_history",
        )

        max_attempts = 3
        for attempt in range(max_attempts):
            try:
                # Agent 실행
                result = agent_with_history.invoke(
                    {"input": state["question"], 
                     "retrieved_data": state.get("context"), 
                     "relevance": state.get("relevance")},
                    {'configurable': {'session_id': state["session_id"]}}
                )

                # 결과에서 코드 실행이 필요하면 safe_code_executor 사용
                # tools 내부에서 자동 호출됨
                state['answer'] = result['output']
                return state

            except Exception as e_inner:
                print(f"⚠️ 에이전트 시도 {attempt+1}/{max_attempts} 실패: {e_inner}")
                if attempt == max_attempts - 1:
                    raise

    except Exception as e:
        print(f"❌ 에이전트 실행 최종 실패: {e}")
        state['answer'] = f"죄송합니다. 질문 처리 중 오류가 발생했습니다: {str(e)[:100]}"
        return state

In [17]:
########################################################################
############################ Workflow Graph ############################
########################################################################

workflow = StateGraph(GraphState)

workflow.add_node("Router", router)
workflow.add_node("Agent", agent)

workflow.add_edge("Router", "Agent")
workflow.add_edge("Agent", END)

workflow.set_entry_point("Router")

memory = MemorySaver()
graph = workflow.compile(checkpointer=memory)  

In [18]:
config = RunnableConfig(recursion_limit=20, configurable={"session_id": "1", "thread_id": uuid.uuid4().hex})  

question = (
    "외국인 근로자가 많은 공장은 어디야? 업종 특징도 알려줘"
)

inputs = GraphState(
    question=question,
    session_id='aaa',
    q_type='',
    answer='',
)

answer = graph.invoke(inputs, config=config)
answer

🆕 새로운 세션 히스토리 생성: aaa...
🆕 새로운 세션 히스토리 생성: 111...


{'question': '외국인 근로자가 많은 공장은 어디야? 업종 특징도 알려줘',
 'q_type': 'domain_specific',
 'answer': '다음 내용은 제공된 공장 데이터(서울 소재)를 분석한 결과입니다.\n\n1. **외국인 근로자가 많은 공장 TOP 10 (인원 기준)**  \n   - (1) **(주)지구화학**  \n     - 공장관리번호: 260210101881100  \n     - 위치: 서울특별시 성동구  \n     - 업종: 사무 및 회화용품 제조업  \n     - 종업원합계: 426명  \n     - 외국인 근로자: 213명 (비율 약 **50%**)  \n\n   - (2) **(주)영일프레시젼**  \n     - 공장관리번호: 115452019436985  \n     - 위치: 서울특별시 금천구  \n     - 업종: 그 외 기타 전자부품 제조업  \n     - 종업원합계: 501명  \n     - 외국인 근로자: 84명 (비율 약 **16.8%**)  \n\n   - (3) **(주)필로스주얼리**  \n     - 공장관리번호: 111102001023720  \n     - 위치: 서울특별시 종로구  \n     - 업종: 귀금속 및 관련제품 제조업  \n     - 종업원합계: 63명  \n     - 외국인 근로자: 63명 (비율 **100%**)  \n\n   - (4) **(주)부래당**  \n     - 공장관리번호: 110111051524101  \n     - 위치: 서울특별시 성동구  \n     - 업종: 여자용 겉옷 제조업  \n     - 종업원합계: 1,092명  \n     - 외국인 근로자: 36명 (비율 약 **3.3%**)  \n\n   - (5) **(주)제이시스메디칼**  \n     - 공장관리번호: 115452004078321  \n     - 위치: 서울특별시 금천구  \n     - 업종: 그 외 기타 의료용 기기 제조업  \n     - 종업원합계: 1,11

In [3]:
df = pd.read_csv('data/cleaned_전국공장등록현황_preprocessed_seoul.csv')

/var/folders/3k/09hbppyx33z8tmqmvw42bgqm0000gn/T/ipykernel_76626/2030280934.py:1: DtypeWarning: Columns (11,13,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/cleaned_전국공장등록현황_preprocessed_seoul.csv')


In [6]:
df.columns

Index(['__row_uid__', '시도명', '시군구명', '관리기관', '회사명', '공장구분', '단지명', '설립구분',
       '입주형태', '보유구분', '최초승인일', '최초등록일', '등록구분', '등록일', '전화번호', '남자종업원',
       '여자종업원', '외국인남자종업원', '외국인여자종업원', '종업원합계', '생산품', '원자재', '공장규모', '용도지역',
       '지목', '용지면적', '제조시설면적', '부대시설면적', '건축면적', '지식산업센터명', '대표업종', '업종명',
       '업종코드', '차수', '법인주소', '필지수', '공장주소', '공장주소_지번', '공장관리번호', '정제_관리기관',
       '코드매핑_관리기관', '누적_관리기관', '정제_대표업종', '코드매핑_대표업종', '누적_대표업종', '정제_등록일',
       '누적_등록일', '정제_보유구분', '코드매핑_보유구분', '누적_보유구분', '정제_시군구명', '코드매핑_시군구명',
       '누적_시군구명', '정제_시도명', '코드매핑_시도명', '누적_시도명', '정제_업종명', '누적_업종명',
       '정제_업종코드', '코드매핑_업종코드', '누적_업종코드', '정제_용도지역', '코드매핑_용도지역', '누적_용도지역',
       '정제_지목', '코드매핑_지목', '누적_지목', '정제_최초등록일', '누적_최초등록일', '정제_최초승인일',
       '누적_최초승인일'],
      dtype='object')

In [4]:
print(answer['answer'])

질문하신 “외국인 근로자가 많은 공장”과 “업종 특징”을, 제공된 데이터 기준으로 정리해 드릴게요.

---

## 1. 외국인 근로자가 많은 공장 (상위 사례)

외국인 비율(외국인 수 / 전체 종업원 수)이 높은 공장 기준 상위 공장들입니다.

### 1) 외국인 근로자 100%인 공장들
(전체 종업원이 모두 외국인인 경우)

- (주)필로스주얼리  
  - 위치: 서울특별시 종로구  
  - 업종: 귀금속 및 관련제품 제조업 (코드 33110)  
  - 종업원 63명 전원이 외국인

- 신흥트래픽(주)  
  - 위치: 서울특별시 은평구  
  - 업종: 전기경보 및 신호장치 제조업 (코드 28901)  
  - 종업원 18명 전원이 외국인

- (주)자바네트웍스  
  - 위치: 서울특별시 금천구  
  - 업종: 방송장비 제조업 (코드 26421)  
  - 종업원 18명 전원이 외국인

- 신광프린테크  
  - 위치: 서울특별시 영등포구  
  - 업종: 기타 인쇄업 (코드 18119)  
  - 종업원 12명 전원이 외국인

- 푸름켐텍  
  - 위치: 서울특별시 금천구  
  - 업종: 물질 검사·측정·분석기구 제조업 (코드 27213)  
  - 종업원 3명 전원이 외국인  

이들 공장은 규모는 크지 않지만, 특정 기술·제조 공정에 외국인 인력이 전적으로 투입된 형태입니다.

### 2) 외국인 비율이 50% 이상인 주요 공장들

- (주)정정에이스  
  - 위치: 서울특별시 금천구  
  - 업종: 도시락류 제조업 (10701)  
  - 외국인 12명 / 전체 15명 (비율 80%)

- (주)정호 어페럴  
  - 위치: 서울특별시 동대문구  
  - 업종: 남자용 겉옷 제조업 (14111)  
  - 외국인 24명 / 전체 36명 (비율 약 66.7%)

- 한올섬유  
  - 위치: 서울특별시 강북구  
  - 업종: 스타킹 및 기타 양말 제조업 (14411)  
  - 외국인 6명 / 전체 9명 (비율 약 66.7%)

- 

In [19]:
config = RunnableConfig(recursion_limit=20, configurable={"session_id": "1", "thread_id": uuid.uuid4().hex})  

question = (
    "서울의 전자부품 업종의 공장수를 알려줘"
)

inputs = GraphState(
    question=question,
    session_id='aaa',
    q_type='',
    answer='',
)

answer = graph.invoke(inputs, config=config)
answer

{'question': '서울의 전자부품 업종의 공장수를 알려줘',
 'q_type': 'domain_specific',
 'answer': '서울 지역에서 **업종에 ‘전자부품’이 포함된 공장 수는 총 295개**입니다.',
 'session_id': 'aaa'}

In [1]:
from main import *

/Users/jeongyunl/Documents/GitHub/factory-chatbot-demo/backend/main.py:58: DtypeWarning: Columns (11,13,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/cleaned_전국공장등록현황_preprocessed_seoul.csv')


In [2]:
config = RunnableConfig(recursion_limit=20, configurable={"session_id": "1", "thread_id": uuid.uuid4().hex})  

question = (
    "서울에서 공장이 가장 많은 구는 어디야?"
)

inputs = GraphState(
    question=question,
    session_id='aaa',
    q_type='',
    answer='',
)

answer = graph.invoke(inputs, config=config)
answer

🆕 새로운 세션 히스토리 생성: aaa...
🆕 새로운 세션 히스토리 생성: session_...


Error in RootListenersTracer.on_chain_end callback: ValueError("Expected str, BaseMessage, List[BaseMessage], or Tuple[BaseMessage]. Got {'q_type': 'domain_specific', 'question': '서울에서 공장이 가장 많은 구는 어디야?'}.")


{'question': '서울에서 공장이 가장 많은 구는 어디야?',
 'q_type': 'domain_specific',
 'answer': '데이터에 따르면 서울에서 공장이 가장 많은 구는 **금천구**이며, 공장 수는 **4,059개**입니다.',
 'session_id': 'aaa'}

In [2]:
config = RunnableConfig(recursion_limit=20, configurable={"session_id": "1", "thread_id": uuid.uuid4().hex})  

question = (
    "최근 5년간 서울 전체 공장 등록 건수 추이를 보여줘"
)

inputs = GraphState(
    question=question,
    session_id='aaa',
    q_type='',
    answer='',
)

answer = graph.invoke(inputs, config=config)
answer

🆕 새로운 세션 히스토리 생성: aaa...
🆕 새로운 세션 히스토리 생성: session_...


Error in RootListenersTracer.on_chain_end callback: ValueError("Expected str, BaseMessage, List[BaseMessage], or Tuple[BaseMessage]. Got {'question': '최근 5년간 서울 전체 공장 등록 건수 추이를 보여줘', 'q_type': 'domain_specific'}.")


{'question': '최근 5년간 서울 전체 공장 등록 건수 추이를 보여줘',
 'q_type': 'domain_specific',
 'answer': '최근 5년(오늘 기준 5년 전부터) 동안 서울특별시 전체 공장 등록 건수 추이는 다음과 같습니다.\n\n- 2020년: 56개\n- 2021년: 653개\n- 2022년: 551개\n- 2023년: 436개\n- 2024년: 495개  \n- 2025년: 357개 (진행 중 연도, 현재까지 집계치)\n\n※ 2025년은 아직 연도가 끝나지 않아, 향후 실제 연간 등록 건수는 더 늘어날 수 있습니다.',
 'session_id': 'aaa'}

# ver2

In [1]:
from langchain_core.runnables import RunnableConfig
import uuid
from main import graph, GraphState

config = RunnableConfig(
    recursion_limit=10,
    configurable={
        "thread_id": f"TEST-{uuid.uuid4().hex[:8]}",
        "session_id": "test_session_ver2"
    }
)

/Users/jeongyunl/Documents/GitHub/factory-chatbot-demo/backend/main.py:30: DtypeWarning: Columns (11,13,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/cleaned_전국공장등록현황_preprocessed_seoul.csv')


In [2]:
question = "서울에서 공장이 가장 많은 구는 어디야?"

inputs = GraphState(
    question=question,
    session_id="test_session_ver2",
    q_type='',
    answer='',
    # context=None,
    # relevance=None,
    execution_id=None,
)

result = graph.invoke(inputs, config=config)
print("답변:", result["answer"])

🆕 새로운 세션 히스토리 생성: test_ses...
🆕 새로운 세션 히스토리 생성: seoul_fa...
답변: [DATA]보유한 공장 현황 데이터를 분석한 결과,  

- **서울에서 공장이 가장 많은 구:** **금천구**  
- **공장 수:** **3,809개**  

분석 과정은 다음과 같습니다.  
1. 데이터에서 **시도명이 ‘서울특별시’인 행만 필터링**했습니다.  
2. 업종명에 **‘제조’ 또는 ‘공장’**이 포함된 사업체를 공장으로 간주해 추려냈습니다.  
3. 이렇게 추려낸 공장들에 대해 **자치구(정제_시군구명)별로 공장 수를 집계**했습니다.  
4. 그 결과, **금천구가 3,809개로 가장 많은 공장 수를 보유한 구**로 나타났습니다.


In [3]:
result

{'question': '서울에서 공장이 가장 많은 구는 어디야?',
 'q_type': 'domain_specific',
 'answer': '[DATA]보유한 공장 현황 데이터를 분석한 결과,  \n\n- **서울에서 공장이 가장 많은 구:** **금천구**  \n- **공장 수:** **3,809개**  \n\n분석 과정은 다음과 같습니다.  \n1. 데이터에서 **시도명이 ‘서울특별시’인 행만 필터링**했습니다.  \n2. 업종명에 **‘제조’ 또는 ‘공장’**이 포함된 사업체를 공장으로 간주해 추려냈습니다.  \n3. 이렇게 추려낸 공장들에 대해 **자치구(정제_시군구명)별로 공장 수를 집계**했습니다.  \n4. 그 결과, **금천구가 3,809개로 가장 많은 공장 수를 보유한 구**로 나타났습니다.',
 'session_id': 'test_session_ver2',
 'execution_id': '16a02a04-817e-46e2-8a43-da567618675c'}